In [1]:
import sqlite3, pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [2]:
# Connect to SQLite database
DB_PATH = "/home/daniel/Downloads/reviews_sample.db"
conn = sqlite3.connect(DB_PATH)

print(f"Connected to {DB_PATH}")


Connected to /home/daniel/Downloads/reviews_sample.db


In [3]:
# Quick check: available tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Available tables:")
print(tables)

Available tables:
              name
0  sqlite_sequence
1          authors
2           hotels
3          reviews
